In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
dest_path ='data/drosophila_enhancer_activity'
split='test'
targets_file = os.path.join(dest_path, 'Sequences_activity_'+split+".txt")
data = pd.read_csv(targets_file, sep="\t")[
            ["Dev_log2_enrichment", "Hk_log2_enrichment"]
        ]
fasta_file= os.path.join(dest_path, 'Sequences_'+split+".fa")
all_seqs = []
all_labels = []

with open(fasta_file) as fin:
    header = False
    for line in fin:
        l = line.strip()
        if len(l) == 0:  # last line
            break
        if line.startswith(">"):
            header = True
            continue
        else:
            assert header  # check fasta format is correct
            all_seqs.append(l)
        
        
        

all_labels=data[["Dev_log2_enrichment", "Hk_log2_enrichment"]].values.astype("float32").tolist()
        
assert len(all_seqs) == len(
            all_labels
        ), "Number of targets does not match number of sequences"
batch_size=128




restrict = lambda x: (
                    torch.cumsum(x, dim=-2)
                    / torch.arange(
                        1, 1 + x.size(-2), device=x.device, dtype=x.dtype
                    ).unsqueeze(-1)
                )[..., -1:, :]   
from scipy.stats import pearsonr
from scipy import stats
def pearsonr_1(outs, y, len_batch=None):
    # TODO: generalize, currently for Monash dataset
    metrics = {}
    outs=outs.detach()
    for i, label in enumerate(['dev', 'hk']):
        y_true = y[:, i].cpu().numpy()
        p = outs[:, i].cpu().numpy()
        r = stats.pearsonr(y_true, p)[0]
        metrics[f'pearsonr_{label}'] = r
        metrics[f'pearsonr2_{label}'] = r ** 2
    metrics['pearsonr'] = (metrics['pearsonr_dev'] + metrics['pearsonr_hk']) / 2
    return metrics
import torch.nn.functional as F
def mse(outs, y, len_batch=None):
    # assert outs.shape[:-1] == y.shape and outs.shape[-1] == 1
    # outs = outs.squeeze(-1)
    # if len(y.shape) < len(outs.shape):
    #     assert outs.shape[-1] == 1
    #     outs = outs.squeeze(-1)
    if len_batch is None:
        # return F.mse_loss(outs, y)
        loss = (
                            (
                                outs[~torch.isnan(y)]
                                - y[~torch.isnan(y)]
                            )
                            ** 2
                        ).mean()
        #check if y include nan
        if torch.isnan(outs).any():
            nan_indices_outs=torch.nonzero(torch.isnan(outs), as_tuple=False)
            print(nan_indices_outs)
        
        if torch.isnan(y).any():
            nan_indices=torch.nonzero(torch.isnan(y), as_tuple=False)
            print(nan_indices)
        return loss
    else:
        # Computes the loss of the first `lens` items in the batches
        # TODO document the use case of this
        mask = torch.zeros_like(outs, dtype=torch.bool)
        for i, l in enumerate(len_batch):
            mask[i, :l, :] = 1
        outs_masked = torch.masked_select(outs, mask)
        y_masked = torch.masked_select(y, mask)
        return F.mse_loss(outs_masked, y_masked)

In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn

max_length=128

with torch.no_grad():
    state_dict='/weight/hyenadna/hyenadna-large-1m-seqlen'
    hyena_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    hyena_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('/outputs/2024-05-04/13-01-30-043019/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    hyena_decoder = nn.Linear(256,2).to('cuda')
    #edit key name in hyena_decoder
    
    hyena_model.load_state_dict(checkpoint,strict=False)
    hyena_decoder.load_state_dict(checkpoint,strict=False)
    hyena_model.eval()
    hyena_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):
        sequence_encoded=hyena_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=hyena_model(input_ids=seqs).last_hidden_state
        hidden_states=restrict(hidden_states)
        out1=hyena_decoder(hidden_states)
        out1_hyena=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_hyena)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            
            if i%1000==0:
                print(i)
                pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn

def group_by_kmer(seq: str, kmer: int) -> str:
        return " ".join(seq[i : i + kmer] for i in range(0, len(seq), kmer)).upper()
max_length=128
with torch.no_grad():
    state_dict='/weight/dnabert/dnabert3/3-new-12w-0'
    bert_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    bert_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('/outputs/2024-03-29/04-45-15-121292/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    bert_decoder = nn.Linear(768,2).to('cuda')
    #edit key name in hyena_decoder
    
    bert_model.load_state_dict(checkpoint,strict=False)
    bert_decoder.load_state_dict(checkpoint,strict=False)
    bert_model.eval()
    bert_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):
        all_seqs_group = group_by_kmer(all_seqs[i],kmer=3)
        sequence_encoded=bert_tokenizer(all_seqs_group,
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=bert_model(input_ids=seqs).last_hidden_state
        hidden_states=restrict(hidden_states)
        out1=bert_decoder(hidden_states)
        out1_bert=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_bert)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn

def group_by_kmer(seq: str, kmer: int) -> str:
        return " ".join(seq[i : i + kmer] for i in range(0, len(seq), kmer)).upper()
max_length=128
with torch.no_grad():
    state_dict='/weight/dnabert2'
    bert2_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    bert2_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('outputs/2024-03-29/08-13-54-405523/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    bert2_decoder = nn.Linear(768,2).to('cuda')
    #edit key name in hyena_decoder
    
    bert2_model.load_state_dict(checkpoint,strict=False)
    bert2_decoder.load_state_dict(checkpoint,strict=False)
    bert2_model.eval()
    bert2_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=bert2_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=bert2_model(input_ids=seqs,export_hidden_states=True)[0]
        hidden_states=restrict(hidden_states)
        out1=bert2_decoder(hidden_states)
        out1_bert2=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_bert2)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
                pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr


In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn

def group_by_kmer(seq: str, kmer: int) -> str:
        return " ".join(seq[i : i + kmer] for i in range(0, len(seq), kmer)).upper()
max_length=128
with torch.no_grad():
    state_dict='/weight/genalm/gena-lm-bert-large-t2t'
    genalm_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    genalm_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('/outputs/2024-03-30/01-49-47-161996/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    genalm_decoder = nn.Linear(1024,2).to('cuda')
    #edit key name in hyena_decoder
    
    genalm_model.load_state_dict(checkpoint,strict=False)
    genalm_decoder.load_state_dict(checkpoint,strict=False)
    genalm_model.eval()
    genalm_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=genalm_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=genalm_model(input_ids=seqs, output_hidden_states=True,).hidden_states[-1]
        hidden_states=restrict(hidden_states)
        out1=genalm_decoder(hidden_states)
        out1_genalm=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_genalm)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
    pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
    print(pearsonr)
            

#plot the bar plot of the pearsonr


In [ ]:
from transformers import AutoTokenizer,AutoModel,AutoModelForMaskedLM
import torch
from torch import nn

def group_by_kmer(seq: str, kmer: int) -> str:
        return " ".join(seq[i : i + kmer] for i in range(0, len(seq), kmer)).upper()
max_length=128
with torch.no_grad():
    state_dict='/weight/nt/nucleotide-transformer-v2-500m-multi-species'
    nt_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    nt_model=AutoModelForMaskedLM.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('/outputs/2024-03-29/10-42-51-953352/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    nt_decoder = nn.Linear(1024,2).to('cuda')
    #edit key name in hyena_decoder
    
    nt_model.load_state_dict(checkpoint,strict=False)
    nt_decoder.load_state_dict(checkpoint,strict=False)
    nt_model.eval()
    nt_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=nt_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=nt_model(input_ids=seqs,output_hidden_states=True)['hidden_states'][-1]
        hidden_states=restrict(hidden_states)
        out1=nt_decoder(hidden_states)
        out1_nt=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_nt)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
    pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
    print(pearsonr)
            

#plot the bar plot of the pearsonr


In [ ]:
from transformers import AutoTokenizer,AutoModel,AutoModelForMaskedLM
import torch
from torch import nn

def group_by_kmer(seq: str, kmer: int) -> str:
        return " ".join(seq[i : i + kmer] for i in range(0, len(seq), kmer)).upper()
max_length=128
with torch.no_grad():
    state_dict='/weight/mamba/caduceus-ph_seqlen-131k_d_model-256_n_layer-16'
    mamba_tokenizer=AutoTokenizer.from_pretrained(state_dict, trust_remote_code=True)
    mamba_model=AutoModel.from_pretrained(state_dict, trust_remote_code=True).to('cuda')
    full_sequence=[]
    checkpoint=torch.load('/outputs/2024-04-19/14-39-48-177210/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "decoder.0.output_transform."
        )

    mamba_decoder = nn.Linear(256,2).to('cuda')
    #edit key name in hyena_decoder
    
    mamba_model.load_state_dict(checkpoint,strict=False)
    mamba_decoder.load_state_dict(checkpoint,strict=False)
    mamba_model.eval()
    mamba_decoder.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):

        sequence_encoded=mamba_tokenizer(all_seqs[i],
                            add_special_tokens= False,  # this is what controls adding eos
                            padding="max_length",
                            max_length=max_length,
                            truncation=True,
                        )
        seq_ids=sequence_encoded['input_ids']
        seq_ids = torch.LongTensor(seq_ids)
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length)).to('cuda')
        target_list.append(target)
        hidden_states=mamba_model(seqs,output_hidden_states=True).last_hidden_state
        hidden_states=restrict(hidden_states)
        out1=mamba_decoder(hidden_states)
        out1_mamba=out1.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_mamba)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            if i%1000==0:
                print(i)
    pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
    print(pearsonr)
            

#plot the bar plot of the pearsonr


In [ ]:
from transformers import AutoTokenizer,AutoModel
import torch
from torch import nn
import sys

from src.models.sequence.deepSTAR import DeepSTAR
max_length=128
def genomic_to_one_hot(genomic_sequence):
        mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
        one_hot = np.zeros((len(genomic_sequence), 4))
        for i, base in enumerate(genomic_sequence):
            if base in mapping:
                one_hot[i, mapping[base]] = 1
            else:
                # 如果碱基不是A、C、G、T或N，可以选择将其编码为全零向量或者平均分配概率
                one_hot[i, :] = 0.25  # 或者使用 np.full((5,), 0.2) 平均分配概率
        return one_hot
with torch.no_grad():
    DeepSTAR_model=DeepSTAR(input_size=128,output_size=2)
    full_sequence=[]
    checkpoint=torch.load('outputs/2024-05-13/09-21-43-779050/checkpoints/val/pearsonr.ckpt')['state_dict']
    torch.nn.modules.utils.consume_prefix_in_state_dict_if_present(
            checkpoint, "model.backbone."
        )
    

    #edit key name in hyena_decoder
    
    DeepSTAR_model.load_state_dict(checkpoint,strict=False)
    DeepSTAR_model.to('cuda')
    DeepSTAR_model.eval()

    target_list=[]
    seq_list=[]
    for i in range(len(all_seqs)):
        seq=genomic_to_one_hot(all_seqs[i])
        seq_ids = torch.from_numpy(seq).float()[:max_length,:]
        target = all_labels[i]
        
        seqs=torch.reshape(seq_ids,(1,max_length,-1)).to('cuda')
        target_list.append(target)
        hidden_states=DeepSTAR_model(seqs)
        out1_deepstar=hidden_states.squeeze(1).squeeze(0).cpu().detach().numpy()
        
        seq_list.append(out1_deepstar)
        seq_list_tensor=torch.FloatTensor(seq_list)
        target_list_tensor=torch.FloatTensor(target_list)
        #calculate the 
        if i>=1:
            
            
            if i%1000==0:
                print(i)
                pearsonr=pearsonr_1(seq_list_tensor,target_list_tensor)
                print(pearsonr)
            

#plot the bar plot of the pearsonr
